# Projet BIML

Bonhoure Timothé 11931551 et Martinez Christophe 11709105

## Partie 1 : Perceptron

**Indiquer et expliquer la taille de chaque tenseur dans le fichier perceptron pytorch.py fourni.**

- $w$ est de taille $784 \times 10$. $784$ est le nombre de pixels dans chaque image ($28 \times 28$) et donc la taille de l'entrée et $10$ est la taille des vecteurs de label et de la sortie.  
- $b$ est de taille $1 \times 10$ car c'est le biais qui est de même dimension que la sortie.  
- $data\_train$ est de taille $784 \times nb\_data\_train$, $nb\_data\_train$ est le nombre d'images dans le set d'entrainement.  
- $data\_test$ est de taille $784 \times nb\_data\_test$, $nb\_data\_test$ est le nombre d'images dans le set de test.  
- $x$ est un batch de $data\_train$ ($784 \times batch\_size$)
- $y$ est la sortie associé au batch $x$ ($10 \times batch\_size$)
- $t$ est un batch des labels, associés à $x$ ($10 \times batch\_size$)
- $grad$ est le tenseur des gradient et est de même dimensions que $t$ et $y$

## Partie 2 : Shallow network

Implémentation de l’algorithme du perceptron multi-couches avec une seule couche cachée et une sortie linéaire.  
L'objectif est de trouver les hyper-paramètres $\eta$ et le nombre de neurones de la couche cachée ($N$).

### Méthodologie

```python
class ShallowNetwork(nn.Module):
	def __init__(self, N) -> None:
		super().__init__()
		self.linear1 = nn.Linear(784, N)
		nn.init.uniform_(self.linear1.weight, -0.001, 0.001)
		self.linear2 = nn.Linear(N, 10)
		nn.init.uniform_(self.linear2.weight, -0.001, 0.001)

	def forward(self, x):
		x = self.linear1(x)
		x = F.relu(x)
		return self.linear2(x)
```

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Lire les données CSV
data = pd.read_csv('results_shallow.csv')

# Séparer les données en fonction du taux d'apprentissage
learning_rates = data['learning_rate'].unique()

# Créer une figure avec deux sous-graphiques
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Parcourir les taux d'apprentissage et créer les graphiques
for lr in learning_rates:
    subset = data[data['learning_rate'] == lr]
    ax1.plot(subset['number_of_neurones'], 1-subset['result'], label=f'LR {lr}')
    ax2.plot(subset['number_of_neurones'], subset['time'], label=f'LR {lr}')

# Configurer le premier graphique (Résultats)
ax1.set_xlabel('Nombre de neurones')
ax1.set_ylabel('Taux d\'erreur')
ax1.set_yscale("log")
yticks = [0.01 * i for i in range(1, 11)] + [0.1 * i for i in range(1, 5)]
ax1.set_yticks(yticks)
ax1.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.set_title('Taux d\'erreur en fonction du nombre de neurones')
ax1.legend()
ax1.grid(True)

# Configurer le deuxième graphique (Temps d'exécution)
ax2.set_xlabel('Nombre de neurones')
ax2.set_ylabel('Temps d\'exécution (s)')
ax2.set_title('Temps d\'exécution en fonction du nombre de neurones')
ax2.legend()
ax2.grid(True)

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher les graphiques
plt.show()

![shallow_results.png](img/shallow_results.png)

### Influence des hyperparamètres

On observe que le nombre de neurones n'a plus d'effet sur le pourcentage de réussite du modèle à partir de 300 à 500 neurones. En revanche, le learning_rate est le paramètre qui a le plus d'effet globalement.  
En terme de temps d'exécution, on observe un pic que l'on ne saurait expliquer pour un nombre de neurones de 100. En dehors de cela, le nombre de neurones à une grande influence sur le temps d'exécution et ceci est cohérent car plus de neurones signifie plus de nombre de modification des poids. Les différents learning_rate sont globalement proche en terme de temps d'exécution mais diffèrent pour un nombre de neurones autours de 200.

Sur 1000 epochs, on voit d'après la courbe ci-dessus à gauche le meilleur résultat est obtenu avec un learning_rate est de 0.01 et pour un nombre de neurone de minimum 500. Or si on s'intéresse au temps d'apprentissage, pour un résultat assez proche et convenable : un learning_rate de 0.001 avec un nombre de neurones entre 200 et 500 est l'optimal.

## Partie 3 : Deep network

On cherche à déterminer les hyperparamètres $\eta$, le nombre de couches ($N_C$), le nombre de neurones pour chaque couche ($N_i$ avec $i \in [\![1,N_C]\!]$) et la taille des batch ($batch\_size$).

### Méthodologie

```python

```

Nous avons décidé de limiter les choix des hyperparamètres :
- $\eta \in \{0.01, 0.001, 0.0001\}$ 
- $N_C \in \{2,3,4\}$
- $N_i \in \{10, 50, 100\}$
- $batch\_size \in \{5,10,15\}$

Ces limites ont été réfléchi en prenant en compte une limitation en puissance de calcul et en temps.

résultat grid search incrementing_factor : `0.9871428608894348`

résultat grid_search first_layer_size :`0.9872857332229614`

## Partie 4 : CNN

L'objectif est d'implémenter un réseau de neurones convolutif.

### Méthodologie

```python
class LeNet5(nn.Module):
	def __init__(self):
		super().__init__()
		self.layer1 = nn.Sequential(
			nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
			nn.BatchNorm2d(6),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size = 2, stride = 2))
		self.layer2 = nn.Sequential(
			nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
			nn.BatchNorm2d(16),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size = 2, stride = 2))
		self.linear1 = nn.Linear(400, 120)
		nn.init.uniform_(self.linear1.weight, -0.001, 0.001)
		self.linear2 = nn.Linear(120, 84)
		nn.init.uniform_(self.linear2.weight, -0.001, 0.001)
		self.linear3 = nn.Linear(84, 10)
		nn.init.uniform_(self.linear3.weight, -0.001, 0.001)
		
	def forward(self, x):
		x = x.view(x.size(0),1,28,28)
		x = self.layer1(x)
		x = self.layer2(x)
		x = x.reshape(x.size(0), -1)
		x = self.linear1(x)
		x = F.relu(x)
		x = self.linear2(x)
		x = F.relu(x)
		x = self.linear3(x)
		return x
```

L'architecture du réseau est inspirée de celle donnée dans [Lecun](https://hal.science/hal-03926082/document).  
La `layer1` exécute la 1ère convolution(`nn.Conv2d`), elle prend en entré un tensor de dimension $(batch\_size, 1, 28, 28)$, le kernel étant de taille 5, cette convolution nécessitait un padding de 2 pour conserver en sortie des matrices de dimension $(28,28)$. Ensuite, `nn.MaxPool2d` applique le subsampling de matrices de $(28,28)$ à des matrices de $(14,14)$.  
La `layer2` exécute la 2ème convolution prenant les matrices précédentes pour sortir 16 matrices de $(10,10)$. Le subsampling suivant transforme les matrices en dimension $(5,5)$.  
Ensuite, des couches similaires au perceptron multi-couches permettent de réduire les résultats en un vecteur de 10 valeurs en faisant succéder une couche linéaire, d'une relu, linéaire, relu et enfin linéaire.

Résultats (à gauche l'accuracy sur le batch de test et à droite le temps d'exécution accumulé en secondes) :
> Sur mon ordinateur
```python
tensor([0.1996]) 50.06423878669739
tensor([0.9476]) 98.38135719299316
tensor([0.9780]) 149.69705057144165
tensor([0.9816]) 215.07379031181335
tensor([0.9851]) 268.4216022491455
tensor([0.9834]) 325.4106557369232
tensor([0.9863]) 376.84096574783325
tensor([0.9850]) 429.86590003967285
tensor([0.9847]) 481.203097820282
tensor([0.9841]) 532.2355885505676
```
> Sur kaggle
```python
tensor([0.4809]) 48.37343430519104
tensor([0.9669]) 94.88335609436035
tensor([0.9789]) 140.88308835029602
tensor([0.9854]) 187.28817486763
tensor([0.9830]) 232.813392162323
tensor([0.9854]) 278.8786988258362
tensor([0.9873]) 324.19943380355835
tensor([0.9873]) 369.5354652404785
tensor([0.9867]) 415.27380990982056
tensor([0.9857]) 461.0540907382965
```

TODO a effacer  
> DeepNetwork :

|$learning\_rate$|0.0001|||0.001|||0.01|||
|---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|$batch\_size$|5|10|15|5|10|15|5|10|15|
||||||||